# Intro
This notebook calculates the force of each player per frame, net force of each team per frame, 
and net partitioned forces per frame

In [1]:
import pandas as pd
import csv
import numpy as np
from datetime import datetime
import dateutil.parser
from IPython.display import display
import math
from math import sin, cos, radians

pd.options.display.max_columns = None

games = pd.read_csv("games.csv")
weeks_dict = dict()
for i in range(1, 9):
    weeks_dict[i] = pd.read_pickle(f'week{i}_clean_pickled.pkl')
pff = pd.read_pickle("pff_pickled.pkl")
pff = pff[['gameId', 'playId', 'nflId', 'pff_role', 'pff_positionLinedUp', 'pff_hit', 'pff_hurry', 'pff_sack']]
players = pd.read_csv("players.csv")
play = pd.read_csv("plays.csv")

# Read in data

Reduce mem

# Force Calculations

In [2]:
# create dictionary to find player weight given their nfl id
players_dict = players[['nflId', 'weight']].set_index('nflId').to_dict()

def find_player_weight(nflID):
    """
    Wrapper function for finding weight of player given their nfl ID.

    Parameters
    ----------
    nflID: int 
        Corresponds to the nfl ID of a given player

    Returns
    -------
    int
        Weight of the player. If the nfl ID does not exist, then return None 
    """
    if nflID in players_dict['weight'].keys():
        return players_dict['weight'][nflID]
    return None


In [3]:
# calculate force and merge with PFF data
merged_weeks = dict()
for i in range(1, 9):
    weeks_dict[i] = weeks_dict[i].drop(columns = ['s', 'o', 'event'])
    weeks_dict[i]['weight'] = weeks_dict[i]['nflId'].apply(lambda x : find_player_weight(x))
    weeks_dict[i]['force'] = weeks_dict[i]['a'] * weeks_dict[i]['weight']
    merged_weeks[i]  = weeks_dict[i].merge(pff, how = 'left', on = ['gameId', 'playId', 'nflId'])


In [4]:
# check if the force is always positive (aka check if no negative acceleration)
for i in range(1, 9):
    mask = (weeks_dict[i]['a'] < 0)
    assert(len(weeks_dict[i][mask]) == 0)

In [5]:
# calculate horizontal and vertical force vectors
# calculations are explained in Kaggle notebook
for i in range(1, 9):
    merged_weeks[i]['adj_angle'] = 90 - merged_weeks[i]['dir'] 
    merged_weeks[i]['adj_angle'] = merged_weeks[i]['adj_angle'] * math.pi / 180
    merged_weeks[i]['cos'] = np.cos(merged_weeks[i]['adj_angle'])
    merged_weeks[i]['sin'] = np.sin(merged_weeks[i]['adj_angle'])
    merged_weeks[i]['x_force'] = merged_weeks[i]['force'] * merged_weeks[i]['cos']
    merged_weeks[i]['y_force'] = merged_weeks[i]['force'] * merged_weeks[i]['sin']
    merged_weeks[i] = merged_weeks[i].drop(columns = ['cos', 'sin'])
    merged_weeks[i].to_csv(f'forces_week{i}.csv', index = False)

# Net Force Calculations

In [6]:
def calculate_net_forces(df):
    """
    Calculates the net x force and y force

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe that contains x force and y force of players over each game, play, and frame

    Returns
    -------
    pd.DataFrame
        Dataframe with net x force and net y force of pass rushers and blockers over each game and play.
        Note that the dataframe is NOT over frame ID
    """
    df_defense = df[df['pff_role'] == 'Pass Rush']
    df_offense = df[df['pff_role'] == 'Pass Block']
    
    offense_grouped = df_offense.groupby(['gameId', 'playId', 'frameId'])
    defense_grouped = df_defense.groupby(['gameId', 'playId', 'frameId'])
    
    net_x_force_offense = offense_grouped['x_force'].sum()
    net_y_force_offense = offense_grouped['y_force'].sum()
    
    net_x_force_defense = defense_grouped['x_force'].sum()
    net_y_force_defense = defense_grouped['y_force'].sum()
    
    x_idx = net_x_force_defense.index.union(net_x_force_offense.index)
    y_idx = net_y_force_defense.index.union(net_y_force_offense.index)
    
    total_x_force = net_x_force_defense.reindex(x_idx, fill_value = 0) + net_x_force_offense.reindex(x_idx, fill_value = 0)
    total_y_force = net_y_force_defense.reindex(y_idx, fill_value = 0) + net_y_force_offense.reindex(y_idx, fill_value = 0)
    
    
    total_x_force_df = total_x_force.to_frame(name = 'net_x_force').reset_index().set_index(['gameId', 'playId', 'frameId'])
    total_y_force_df = total_y_force.to_frame(name = 'net_y_force').reset_index().set_index(['gameId', 'playId', 'frameId'])
    
    total_force_df = pd.concat([total_x_force_df, total_y_force_df], axis = 1).reset_index()
    return total_force_df

In [7]:
# run calculate_net_forces for all our dataframes
net_forces_dict = dict()
for i in range(1, 9):
    net_forces_dict[i] = calculate_net_forces(merged_weeks[i])

# Partitioned Force Calculations

In [8]:
def find_lt_y(df):
    """
    Find the y value of the leftmost pass blocker, which is usually the left tackle. 
    
    If left tackle cannot be found, we return the y value of the left guard.

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe for a given game ID, play ID
        
    Returns
    -------
    float
        y value of leftmost pass blocker 
    """
    lg_y = df[df['pff_positionLinedUp'] == 'LT']['y']
    if len(lg_y) == 0:
        lg_y = lg_y = df[df['pff_positionLinedUp'] == 'LG']['y']
    return lg_y.values[0]

def find_rt_y(df):
    """
    Find the y value of the rightmost pass blocker, which is usually the right tackle. 
    
    If right tackle cannot be found, we return the y value of the right guard.

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe for a given game ID, play ID
        
    Returns
    -------
    float
        y value of rightmost pass blocker 
    """

    rg_y = df[df['pff_positionLinedUp'] == 'RT']['y']
    if len(rg_y) == 0:
        rg_y = df[df['pff_positionLinedUp'] == 'RG']['y']
    return rg_y.values[0]
    

def create_indicator(df, simple = False):
    """
    Create a dataframe with an indicator column that indicates which partition a player is in

    Parameters
    ----------
    df: pd.DataFrame
        Dataframe that includes tracking data of player over game, play, and frame.
    simple: bool
        Optional parameter to specify if partitions stay fixed throughout a play or change over the course of a play.
        If simple == False, then the partitions per frame ID are based on locations of 
        rightmost and leftmost pass blockers.
        
    Returns
    -------
    pd.DataFrame
        y value of leftmost pass blocker 
    """
    if simple:
        to_merge1 = df.groupby(['gameId', 'playId']).apply(lambda x : find_lt_y(x)).reset_index().rename(columns = {0 : 'lt_y'})
        to_merge2 = df.groupby(['gameId', 'playId']).apply(lambda x : find_rt_y(x)).reset_index().rename(columns = {0 : 'rt_y'})
        new_df = pd.merge(df, to_merge1, how = "left", on = ["playId","gameId"])
        new_df = pd.merge(new_df, to_merge2, how = "left", on = ["playId","gameId"])
    else:
        # the two merges take around ~30-60 seconds
        to_merge1 = df.groupby(['gameId', 'playId', 'frameId']).apply(lambda x : find_lt_y(x)).reset_index().rename(columns = {0 : 'lt_y'})
        to_merge2 = df.groupby(['gameId', 'playId', 'frameId']).apply(lambda x : find_rt_y(x)).reset_index().rename(columns = {0 : 'rt_y'})
    
        new_df = pd.merge(df, to_merge1, how="left",on = ["frameId","playId","gameId"])
        new_df = pd.merge(new_df, to_merge2, how="left",on = ["frameId","playId","gameId"])
    
    new_df['max_y'] = new_df[['lt_y', 'rt_y']].max(axis = 1)
    new_df['min_y'] = new_df[['lt_y', 'rt_y']].min(axis = 1)
    new_df['indicator'] = np.where(new_df['y'] < new_df['min_y'], -1, 
                                         np.where((new_df['min_y'] <= new_df['y']) & (new_df['y'] <= new_df['max_y']), 0, 
                                                 1))
    return new_df    

In [9]:
# calculate the partitioned forces
partitioned_dict = dict()
for i in range(1, 9):
    force_with_indicator_df = create_indicator(merged_weeks[i], simple = True)
    net_force_df = force_with_indicator_df.groupby(['indicator']).apply(calculate_net_forces)
    net_force_df = net_force_df.reset_index(level = 1, drop = True)
    net_force_df = net_force_df.reset_index()
    
    # groups will hold dataframe of net force per partitione
    groups = []
    for n, g in net_force_df.groupby('indicator'):
        groups.append(g)
        
    # merge all the partitioned dataframes together
    df = groups[0].merge(groups[1], how = 'outer', on = ['gameId', 'playId', 'frameId'])
    df = df.merge(groups[2], how = 'outer', on = ['gameId', 'playId', 'frameId'])

    # rename columns
    df = df.rename(columns = {'net_x_force' : 'net_x_force_top', 'net_y_force' : 'net_y_force_top',
                             'net_x_force_x' : 'net_x_force_bottom', 'net_y_force_x' : 'net_y_force_bottom',
                             'net_x_force_y' : 'net_x_force_middle', 'net_y_force_y' : 'net_y_force_middle'})
    df = df.drop(columns = ['indicator', 'indicator_x', 'indicator_y'])
    partitioned_dict[i] = df

In [10]:
# merge partitioned net forces and net forces into 1 data frame
for i in range(1, 9):
    df = pd.merge(partitioned_dict[i], net_forces_dict[i], how = 'outer', on = ['gameId', 'playId', 'frameId'])
    df.to_csv(f"partitioned_forces{i}.csv")
    display(df)

,gameId,playId,frameId,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,net_x_force,net_y_force
0,2021090900,97,1,-225.829434,76.702754,-3011.310221,1139.103266,-575.871280,-235.595226,-3813.010935,980.210794
1,2021090900,97,2,-526.695232,-323.384263,-2985.756761,1347.738964,-1868.916410,-71.714197,-5381.368403,952.640505
2,2021090900,97,3,-676.573980,-406.991765,-3901.708203,1432.426128,-1796.744798,-194.362087,-6375.026980,831.072275
3,2021090900,97,4,-752.977317,-363.143410,-3833.114185,1327.202126,-1515.847517,-15.296470,-6101.939019,948.762246
4,2021090900,97,5,-749.841624,-192.021941,-3424.444798,1143.815613,-1349.885166,-150.653130,-5524.171589,801.140542
...,...,...,...,...,...,...,...,...,...,...,...
36660,2021091208,225,65,NaN,NaN,NaN,NaN,-920.950274,3026.761700,-920.950274,3026.761700
36661,2021091208,225,66,NaN,NaN,NaN,NaN,-1092.058652,3545.024487,-1092.058652,3545.024487
36662,2021091208,225,67,NaN,NaN,NaN,NaN,-1204.373459,3947.722359,-1204.373459,3947.722359
36663,2021091208,225,68,NaN,NaN,NaN,NaN,-1261.216845,4290.259973,-1261.216845,4290.259973


,gameId,playId,frameId,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,net_x_force,net_y_force
0,2021091600,65,1,-173.004864,-26.349844,-394.136810,-363.202107,92.685847,30.258604,-474.455827,-359.293347
1,2021091600,65,2,-826.523439,-483.561180,-288.220159,151.129172,68.115401,-86.372983,-1046.628196,-418.804991
2,2021091600,65,3,-1121.892953,-634.063197,-702.374502,538.540615,-79.483012,-207.818912,-1903.750468,-303.341494
3,2021091600,65,4,-1022.767502,-558.327883,-764.866308,538.517683,-324.884624,-177.059241,-2112.518434,-196.869441
4,2021091600,65,5,-948.847805,-442.488353,-882.189217,642.257289,-406.464529,-181.139654,-2237.501551,18.629283
...,...,...,...,...,...,...,...,...,...,...,...
34440,2021091909,1797,54,NaN,NaN,NaN,NaN,1150.063667,3948.026852,1150.063667,3948.026852
34441,2021091909,1797,55,NaN,NaN,NaN,NaN,1063.812190,3737.929648,1063.812190,3737.929648
34442,2021091909,1797,56,NaN,NaN,NaN,NaN,1372.927827,4075.763618,1372.927827,4075.763618
34443,2021091909,1797,57,NaN,NaN,NaN,NaN,1600.100834,4302.049357,1600.100834,4302.049357


,gameId,playId,frameId,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,net_x_force,net_y_force
0,2021092300,54,1,90.817510,100.517833,-168.679371,-36.222879,-37.549749,50.487051,-115.411610,114.782005
1,2021092300,54,2,380.795071,388.088128,-308.980987,728.073583,-230.343495,500.448765,-158.529411,1616.610476
2,2021092300,54,3,737.267939,722.392190,-145.348948,1249.943934,-442.324236,849.847003,149.594755,2822.183126
3,2021092300,54,4,1017.127091,1041.952261,-35.056355,1744.965814,-553.266171,1207.015204,428.804564,3993.933279
4,2021092300,54,5,1326.663563,1222.375447,-39.397163,1722.580422,-581.404337,1171.243698,705.862063,4116.199568
...,...,...,...,...,...,...,...,...,...,...,...
37289,2021092604,940,39,NaN,NaN,NaN,NaN,643.607335,50.090986,643.607335,50.090986
37290,2021092613,2453,26,NaN,NaN,NaN,NaN,-1076.936025,3051.321479,-1076.936025,3051.321479
37291,2021092613,2453,27,NaN,NaN,NaN,NaN,-752.328501,2893.967853,-752.328501,2893.967853
37292,2021092613,2453,28,NaN,NaN,NaN,NaN,-489.961831,2949.088751,-489.961831,2949.088751


,gameId,playId,frameId,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,net_x_force,net_y_force
0,2021093000,169,2,-198.937692,-39.390735,-1890.219751,7.739674,-320.254331,101.652145,-2409.411774,70.001084
1,2021093000,169,3,-464.746442,-55.088591,-2286.430392,-51.859424,-563.754767,219.005976,-3314.931600,112.057961
2,2021093000,169,4,-538.527667,-68.604966,-2899.918023,-118.667506,-617.870641,247.505978,-4056.316331,60.233506
3,2021093000,169,5,-539.925832,-56.557975,-3084.254555,-270.870576,-674.803072,232.090001,-4298.983459,-95.338550
4,2021093000,169,6,-502.671159,-52.832762,-3130.864527,-363.194881,-693.704605,236.156202,-4327.240291,-179.871441
...,...,...,...,...,...,...,...,...,...,...,...
35442,2021100400,4118,46,NaN,NaN,1580.084944,-95.454250,587.413332,-254.561707,2167.498276,-350.015957
35443,2021100400,4139,34,NaN,NaN,-1917.187763,-357.728471,520.411289,135.419201,-1396.776474,-222.309270
35444,2021100400,4139,35,NaN,NaN,-1838.305900,-416.599067,354.843727,295.854548,-1483.462173,-120.744519
35445,2021100400,4139,36,NaN,NaN,-1733.535281,-81.251895,276.492338,219.616944,-1457.042943,138.365048


,gameId,playId,frameId,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,net_x_force,net_y_force
0,2021100700,95,1,-130.614928,-11.611060,-430.264846,-100.536961,NaN,NaN,-560.879774,-112.148021
1,2021100700,95,2,-468.938647,-280.569754,-363.589272,121.662370,NaN,NaN,-832.527919,-158.907384
2,2021100700,95,3,-458.631603,-341.359636,-869.371285,-152.077337,0.000000,0.000000,-1328.002888,-493.436972
3,2021100700,95,4,-557.759338,-337.516249,-1279.526470,-400.057020,0.000000,0.000000,-1837.285809,-737.573269
4,2021100700,95,5,-954.901810,-663.941257,-1266.548306,-110.075453,0.000000,0.000000,-2221.450116,-774.016710
...,...,...,...,...,...,...,...,...,...,...,...
36520,2021101010,2377,52,NaN,NaN,NaN,NaN,1651.942646,5906.023295,1651.942646,5906.023295
36521,2021101010,2377,53,NaN,NaN,NaN,NaN,1560.781642,6230.846099,1560.781642,6230.846099
36522,2021101010,2377,54,NaN,NaN,NaN,NaN,1580.479187,6710.352218,1580.479187,6710.352218
36523,2021101010,2377,55,NaN,NaN,NaN,NaN,1452.453376,6877.142977,1452.453376,6877.142977


,gameId,playId,frameId,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,net_x_force,net_y_force
0,2021101400,76,1,-180.580614,116.778200,-517.100629,317.186883,0.000000,0.000000,-697.681243,433.965082
1,2021101400,76,2,-569.398275,162.090263,-687.025286,150.242319,-212.164428,112.145668,-1468.587988,424.478249
2,2021101400,76,3,-933.540345,13.524659,-660.528051,532.544195,-412.565996,217.613352,-2006.634392,763.682206
3,2021101400,76,4,-1284.051145,-182.783447,-806.847519,882.794989,-539.441842,190.984807,-2630.340506,890.996348
4,2021101400,76,5,-1465.765266,-234.278372,-1012.732676,1206.509608,-839.622557,-188.336818,-3318.120499,783.894418
...,...,...,...,...,...,...,...,...,...,...,...
32233,2021101800,3914,20,NaN,NaN,-1667.298050,611.933996,-1481.947046,-734.618607,-3149.245097,-122.684611
32234,2021101800,3914,21,NaN,NaN,-2166.356346,107.064522,-765.632525,-464.173613,-2931.988872,-357.109091
32235,2021101800,3914,22,NaN,NaN,-2059.556107,-303.911354,-650.789709,120.011584,-2710.345816,-183.899770
32236,2021101800,3914,23,NaN,NaN,-2099.262274,-46.238015,-860.601150,45.606268,-2959.863424,-0.631747


,gameId,playId,frameId,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,net_x_force,net_y_force
0,2021102100,56,1,-4.435054,2.308743,181.947104,-1465.911004,-86.579661,-8.016538,90.932389,-1471.618799
1,2021102100,56,2,-242.820072,-574.588984,270.739282,-1486.519427,-145.416950,-38.393603,-117.497740,-2099.502013
2,2021102100,56,3,-567.175420,-876.391677,154.809979,-2320.037153,-180.740162,-90.428998,-593.105603,-3286.857829
3,2021102100,56,4,-655.286926,-850.063420,-77.168604,-3068.691051,-208.345520,-257.652774,-940.801050,-4176.407246
4,2021102100,56,5,-766.151805,-913.013003,-468.970450,-4648.979581,-255.825630,-435.695815,-1490.947885,-5997.688399
...,...,...,...,...,...,...,...,...,...,...,...
29909,2021102402,483,71,NaN,NaN,NaN,NaN,1587.379272,3494.398366,1587.379272,3494.398366
29910,2021102402,483,72,NaN,NaN,NaN,NaN,1572.519151,3814.486464,1572.519151,3814.486464
29911,2021102402,483,73,NaN,NaN,NaN,NaN,1381.782216,4016.560146,1381.782216,4016.560146
29912,2021102402,483,74,NaN,NaN,NaN,NaN,1242.324164,4413.096778,1242.324164,4413.096778


,gameId,playId,frameId,net_x_force_bottom,net_y_force_bottom,net_x_force_middle,net_y_force_middle,net_x_force_top,net_y_force_top,net_x_force,net_y_force
0,2021102800,189,1,-406.542084,98.353021,-1659.031382,206.143111,-458.083809,-373.337801,-2523.657274,-68.841668
1,2021102800,189,2,-1098.364567,-14.003862,-1293.330136,-151.149217,-1744.043576,-228.171201,-4135.738280,-393.324280
2,2021102800,189,3,-1441.543171,45.219422,-1672.517759,-660.716025,-1816.003827,-187.726547,-4930.064757,-803.223150
3,2021102800,189,4,-1667.232127,46.659996,-1697.216166,-760.688120,-1825.345127,-272.743134,-5189.793420,-986.771258
4,2021102800,189,5,-1766.653047,24.374616,-2040.692405,-737.620367,-1866.620090,-335.774853,-5673.965542,-1049.020604
...,...,...,...,...,...,...,...,...,...,...,...
32106,2021103112,2374,63,NaN,NaN,NaN,NaN,-1559.933707,4317.217329,-1559.933707,4317.217329
32107,2021103112,2374,64,NaN,NaN,NaN,NaN,-1613.862842,4199.975809,-1613.862842,4199.975809
32108,2021103112,2374,65,NaN,NaN,NaN,NaN,-1724.669485,4178.523932,-1724.669485,4178.523932
32109,2021103112,2374,66,NaN,NaN,NaN,NaN,-1776.234654,4144.150868,-1776.234654,4144.150868
